# **EDA AND DATA CLEANING NOTEBOOK**

## Objectives

* Carry Out EDA relating to quality of data
* Clean data and save for use in further analysis

## Inputs

* **Raw Dataset:** inputs/datasets/raw/hotel_bookings.csv

## Outputs

* **Cleaned Dataset:** outputs/datasets/collection/cleaned/cleaned.csv

---

# Import Packages and Load Data

Imports

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

Load data

In [ ]:
from pathlib import Path

project_root = Path.cwd().parent
dataset_file = project_root / 'inputs' / 'datasets' / 'raw' / 'hotel_bookings.csv'
df = pd.read_csv(dataset_file)
df.head(3)

---

# Convert Data Types

View current data types

In [ ]:
df.info()

Convert data types ready for profiling report.
- `Int64` allows for the missing values and is used for `children`, `agent` and `company` to remove decimal digits before converting to `category` type

In [ ]:
df['hotel'] = df['hotel'].astype('category')
df['is_canceled'] = df['is_canceled'].astype('bool')
df['arrival_date_year'] = df['arrival_date_year'].astype('category')
df['arrival_date_month'] = df['arrival_date_month'].astype('category')
df['children'] = df['children'].astype('Int64')
df['meal'] = df['meal'].astype('category')
df['country'] = df['country'].astype('category')
df['market_segment'] = df['market_segment'].astype('category')
df['distribution_channel'] = df['distribution_channel'].astype('category')
df['is_repeated_guest'] = df['is_repeated_guest'].astype('bool')
df['reserved_room_type'] = df['reserved_room_type'].astype('category')
df['assigned_room_type'] = df['assigned_room_type'].astype('category')
df['deposit_type'] = df['deposit_type'].astype('category')
df['agent'] = df['agent'].astype('Int64').astype('category')
df['company'] = df['company'].astype('Int64').astype('category')
df['customer_type'] = df['customer_type'].astype('category')
df['reservation_status'] = df['reservation_status'].astype('category')
df['reservation_status_date'] = pd.to_datetime(df['reservation_status_date'])

df.info()

# Profile Report

In [ ]:
from ydata_profiling import ProfileReport
profile = ProfileReport(df.drop_duplicates(), title="Hotel Bookings Profile Report", minimal=True)
profile.to_notebook_iframe()

The Profile Report alerts us to some possible data issues which require further exploration:
- **Data Ranges:**
  - `lead_time` shows two values over 600
  - `stays_in_weekend_nights` shows some values over 10
  - `stays_in_week_nights` shows some values over 30
  - `adults` shows some values over 20
  - `children` shows a high value of 10
  - `babies` shows high values of 9 and 10
  - `previous_cancellations` shows values over 20
  - `previous_bookings_not_canceled` shows values over 60
  - `booking_changes` shows values over 20
  - `days_in_waiting_list` shows values over 300
  - `adr` shows at least one negative value, many zero values and an incredibly high value of 5400
  - `required_car_parking_spaces` shows a high value of 8
  - `reservation_status_dates` that lie outside of the date range for when guests were due to arrive
- **Undefined Values for Categorical Variables:**
  - there are 'Undefined' values for `meal`, `market_segment`, `distribution_channel`
- **Missing Values**
  - there are missing values for `children`, `country`, `agent` and `company`

---

# Exploring Anomalies and Inconsistencies

## Function to assist analysis

Define function for summarising value counts and percentages in a table

In [186]:
def value_counts_and_percentages(df, filter_by_cols=None):
    data = df[filter_by_cols] if filter_by_cols else df
    df_count = data.value_counts(dropna=False)
    df_percent = round(data.value_counts(normalize=True, dropna=False) * 100, 1)
    summary = pd.concat([df_count, df_percent], axis=1)
    summary.columns = ['Count', '%']
    return summary

## Remove 'Undefined' Values

### Meal

View categories

In [160]:
summary = value_counts_and_percentages(df, ['meal'])
summary

,Count,%
meal,,
BB,92310,77.3
HB,14463,12.1
SC,10650,8.9
Undefined,1169,1.0
FB,798,0.7


Drop 'Undefined'

In [171]:
# Drop rows with values of 'Undefined'
condition = (df['meal'] == 'Undefined')
df = df[~condition]

# Check values have been dropped
summary = value_counts_and_percentages(df, ['meal'])
summary

,Count,%
meal,,
BB,92306,78.1
HB,14462,12.2
SC,10650,9.0
FB,798,0.7
Undefined,0,0.0


### Market Segment

View categories

In [162]:
summary = value_counts_and_percentages(df, ['market_segment'])
summary

,Count,%
market_segment,,
Online TA,56461,47.8
Offline TA/TO,23961,20.3
Groups,19010,16.1
Direct,12533,10.6
Corporate,5280,4.5
Complementary,737,0.6
Aviation,237,0.2
Undefined,2,0.0


Drop 'Undefined'

In [163]:
# Drop rows with values of 'Undefined'
condition = (df['market_segment'] == 'Undefined')
df = df[~condition]

# Check values have been dropped
summary = value_counts_and_percentages(df, ['market_segment'])
summary

,Count,%
market_segment,,
Online TA,56461,47.8
Offline TA/TO,23961,20.3
Groups,19010,16.1
Direct,12533,10.6
Corporate,5280,4.5
Complementary,737,0.6
Aviation,237,0.2
Undefined,0,0.0


### Distribution Channel

View categories

In [164]:
summary = value_counts_and_percentages(df, ['distribution_channel'])
summary

,Count,%
distribution_channel,,
TA/TO,97021,82.1
Direct,14372,12.2
Corporate,6630,5.6
GDS,193,0.2
Undefined,3,0.0


Drop 'Undefined'

In [165]:
# Drop rows with values of 'Undefined'
condition = (df['distribution_channel'] == 'Undefined')
df = df[~condition]

# Check values have been dropped
summary = value_counts_and_percentages(df, ['distribution_channel'])
summary

,Count,%
distribution_channel,,
TA/TO,97021,82.1
Direct,14372,12.2
Corporate,6630,5.6
GDS,193,0.2
Undefined,0,0.0


## Lead Time

Show distribution

In [ ]:
sns.histplot(df, x='lead_time')

Analyse values over 600

In [ ]:
cols = ['hotel', 'lead_time', 'arrival_date_year', 'arrival_date_month', 'arrival_date_week_number', 'stays_in_weekend_nights',
        'stays_in_week_nights', 'meal', 'market_segment', 'distribution_channel', 'agent', 'company','is_repeated_guest',
        'reserved_room_type','assigned_room_type', 'deposit_type', 'days_in_waiting_list',
        'customer_type', 'adr', 'required_car_parking_spaces', 'total_of_special_requests', 'reservation_status', 'reservation_status_date']
cols = df.columns
data = df[df['lead_time'] > 600][cols].value_counts(dropna=False)
pd.DataFrame(data)

The bookings with lead times between 600 and 700 share a number of similarities:
- same hotel (City Hotel)
- same travel agent
- for off-peak seasons
- all group bookings
- same room type and meal plan
- adr within 58-63 range
- resulted in cancelled bookings

The bookings with lead times above 700 also share a number of similarities:
- same hotel (resort hotel)
- direct bookings
- were not cancelled

They are also both unusual in different ways:
- one has no overnight stays (either weekdays or weekends) and an adr of 0
  - NOTE: a deeper analysis of adr=0 follows later
- one has 28 overnight stays (i.e. 4 weeks)

These bookings do not appear to be random anomalies but rather coherent subsets of bookings. They are also not too much higher than other lead time values.

**ACTION:** keep all records

## Overnight Stays

Add a calculated `total_nights` variable to assist in analysis

In [172]:
df_nights = df.copy().drop_duplicates()
df_nights['total_nights'] = df_nights['stays_in_weekend_nights'] + df['stays_in_week_nights']

### Check consistency between `stays_in_weekend_nights` and `stays_in_week_nights`

By calculating the weeks that a guest stayed using `stays_in_weekend_nights` and `stays_in_week_nights`, we can check that the difference is never greater than 1.

In [ ]:
# Create columns for weeks calculated by weekday nights and weekend nights
df_nights['weekend_weeks'] = df_nights['stays_in_weekend_nights'] / 2
df_nights['weekday_weeks'] = df_nights['stays_in_week_nights'] / 5

# If consistent, the difference between weekend_weeks and weekday_weeks should be <= 1
df_nights['weeks_are_consistent'] = (abs(df_nights['weekend_weeks'] - df_nights['weekday_weeks'])<=1)
df_nights['weeks_are_consistent'].value_counts()

All records have consistent values.

### Analyse highest values

Inspect records with long stays

In [ ]:
high_overnight_stays = df_nights[(df_nights['stays_in_weekend_nights']>10) | (df_nights['stays_in_week_nights']>30)]
high_overnight_stays

These observations all seem plausible. For example, there are no babies or children staying for this long. Some of the average daily rates are very low with 4 of the stays being without charge. This is presumably a concession for special guests.

### Zero Overnight Stays

Inspect records with no overnight stays

In [173]:
no_overnight_stays = df_nights[(df_nights['total_nights']==0)]
no_overnight_stays

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,total_nights
0,Resort Hotel,False,342,2015,July,27,1,0,0,2,...,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01,0.0
1,Resort Hotel,False,737,2015,July,27,1,0,0,2,...,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01,0.0
167,Resort Hotel,False,111,2015,July,28,6,0,0,2,...,240,NaN,0,Transient,0.0,0,2,Check-Out,2015-07-06,0.0
168,Resort Hotel,False,0,2015,July,28,6,0,0,1,...,250,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-06,0.0
196,Resort Hotel,False,8,2015,July,28,7,0,0,2,...,NaN,NaN,0,Transient,0.0,0,1,Check-Out,2015-07-07,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115483,City Hotel,False,15,2017,July,27,6,0,0,1,...,14,NaN,0,Transient-Party,0.0,0,0,Check-Out,2017-07-06,0.0
117701,City Hotel,False,0,2017,August,32,8,0,0,2,...,262,NaN,0,Transient,0.0,0,0,Check-Out,2017-08-08,0.0
118029,City Hotel,False,0,2017,August,33,14,0,0,2,...,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2017-08-14,0.0
118631,City Hotel,False,78,2017,August,34,23,0,0,1,...,98,NaN,0,Transient-Party,0.0,0,0,Check-Out,2017-08-23,0.0


It is unclear what these bookings are but they are clearly a special case.

These bookings constitute less than 1% of the overall data (duplicates removed).

In [209]:
summary = value_counts_and_percentages(df_nights, ['total_nights'])
summary.loc[[0.0]]

,Count,%
total_nights,,
0.0,644,0.7


All of these records have **adr = 0**. According to the [original data source](https://www.sciencedirect.com/science/article/pii/S2352340918315191), Average Daily Rates are calculated by
> dividing the sum of all lodging transactions by the total number of staying nights.

If there are no staying nights, the value will be undefined and the system presumably defaults to zero.

In [210]:
summary = value_counts_and_percentages(no_overnight_stays, ['adr'])
summary

,Count,%
adr,,
0.0,644,100.0


Cancellation rates are much lower for this type of booking than overnight stays

In [218]:
condition = df_nights['total_nights'] == 0.0

# Get percentage counts for day-only bookings and overnight bookings
day_only_counts = value_counts_and_percentages(df_nights[condition], ['is_canceled'])
overnight_counts = value_counts_and_percentages(df_nights[~condition], ['is_canceled'])

# Concatenate data into one summary table
summary = pd.concat([day_only_counts, overnight_counts], axis=1)

# Drop Counts and rename columns
summary.drop('Count', axis=1, inplace=True)
summary.columns = ['No Overnight Stays (%)', 'Yes Overnight Stays (%)']

summary

,No Overnight Stays (%),Yes Overnight Stays (%)
is_canceled,,
False,95.5,72.3
True,4.5,27.7


Of all bookings that have adr = 0.0, the majority of these do consist of at least one overnight stay (~63%) but bookings with no overnight stays still constitute a significant minority and therefore will not be removed for the moment.

In [224]:
zero_adr = df_nights[df_nights['adr'] == 0.0]
summary = value_counts_and_percentages(zero_adr, ['total_nights'])
summary

,Count,%
total_nights,,
0.0,644,36.8
1.0,497,28.4
2.0,318,18.2
3.0,108,6.2
4.0,58,3.3
7.0,30,1.7
5.0,28,1.6
6.0,15,0.9
8.0,10,0.6


## Investigating Distribution Channel

### Initial Investigations

View values for `distribution_channel`

In [225]:
summary = value_counts_and_percentages(df, ['distribution_channel'])
display(summary)

,Count,%
distribution_channel,,
TA/TO,97021,82.1
Direct,14372,12.2
Corporate,6630,5.6
GDS,193,0.2
Undefined,0,0.0


**Undefined** values were almost certainly a data entry / system error and have therefore already been dropped.

### `distribution_channel` + `agent`

We expect that when an **agent** is specified, **distribution_channel = TA/TO**

In [226]:
data = df[~df['agent'].isna()]
summary = value_counts_and_percentages(data, ['distribution_channel'])
display(summary)

,Count,%
distribution_channel,,
TA/TO,93957,91.9
Direct,6935,6.8
Corporate,1134,1.1
GDS,191,0.2
Undefined,0,0.0


Findings
- **TA/TO** is expected:
  - This is the majority class with over 90% of all observations
  - ACTION: keep
- **Direct** is inconsistent:
  - Probably `distribution_channel` or `agent` is mis-labelled but impossible to know which.
  - ACTION: drop
- **Corporate** is possible:
  - Some corporate bookings are handled by travel agents (e.g. travel management companies).
  - ACTION: investigate further
- **GDS** is valid:
  - Many travel agents book via GDS.
  - ACTION: keep

Check that similar patterns are seen with both hotels

In [227]:
# City
city_data = df[(~df['agent'].isna() & (df['hotel'] == 'City Hotel'))]
city_summary = value_counts_and_percentages(city_data, ['distribution_channel'])

# Resort
resort_data = df[(~df['agent'].isna() & (df['hotel'] == 'Resort Hotel'))]
resort_summary = value_counts_and_percentages(resort_data, ['distribution_channel'])

# Concatenate into one table and remove counts
summary = pd.concat([city_summary, resort_summary], axis=1)
summary.drop('Count', axis=1, inplace=True)

# Rename column headings and display
summary.columns = ['% of City Hotel Bookings', '% of Resort Hotel Bookings']
summary


,% of City Hotel Bookings,% of Resort Hotel Bookings
distribution_channel,,
TA/TO,93.7,87.7
Direct,5.2,10.4
Corporate,0.8,1.8
GDS,0.3,0.0
Undefined,0.0,0.0


Similar patterns are seen with both hotels so there don't seem to be issues with administrative errors in only one of the hotels.

Investigate `Corporate` further
- How many of the 1134 corporate bookings have a company ID specified alongside the travel agent?

In [228]:
data = df[(~df['agent'].isna()) & (~df['company'].isna())]
summary = value_counts_and_percentages(data, ['distribution_channel'])
display(summary)

,Count,%
distribution_channel,,
Corporate,132,60.8
TA/TO,64,29.5
Direct,20,9.2
GDS,1,0.5
Undefined,0,0.0


Findings
- Only 132 of the 1134 agent bookings assigned distribution_channel = corporate have a company ID associated with them (~12%)
- These may still be valid records where the company was not recorded
- ACTION: keep 'Corporate' but drop 'Direct'


In [ ]:
# Drop rows with specified agent but distribution_channel = 'Direct'
condition = (~df['agent'].isna()) & (df['distribution_channel'] == 'Direct')
df = df[~condition]

data = df[~df['agent'].isna()]
summary = value_counts_and_percentages(data, ['distribution_channel'])
display(summary)

### `distribution_channel` + `company`

We expect that when a **company** is specified, **distribution_channel = Corporate**

In [ ]:
data = df[~df['company'].isna()]
summary = value_counts_and_percentages(data, ['distribution_channel'])
display(summary)

Findings
- **Corporate** is expected:
  - This is the majority class with ~ 75% of all observations
  - ACTION: keep
- **TA/TO** is possible:
  - Some corporate bookings are handled by travel agents (e.g. travel management companies).
  - ACTION: keep
- **Direct** is inconsistent:
  - Probably `distribution_channel` or `company` is mis-labelled but impossible to know which.
  - ACTION: drop
- **GDS** is possible:
  - The company could have booked via GDS.
  - ACTION: keep

In [ ]:
# Drop rows with specified company but distribution_channel = 'Direct'
condition = (~df['company'].isna()) & (df['distribution_channel'] == 'Direct')
df = df[~condition]

data = df[~df['company'].isna()]
summary = value_counts_and_percentages(data, ['distribution_channel'])
display(summary)

## Check `is_canceled` matches `reservation_status`

We expect that records with **is_canceled = 0** should have **reservation_status = 'Check-Out'**

In [229]:
data = df[df['is_canceled']==0]
summary = value_counts_and_percentages(data, ['reservation_status'])
display(summary)

,Count,%
reservation_status,,
Check-Out,74282,100.0
Canceled,0,0.0
No-Show,0,0.0


This is the case. We also expect that records with **is_canceled = 1** should have **reservation_status = 'Canceled' or 'No-Show'**

In [230]:
data = df[df['is_canceled']==1]
summary = value_counts_and_percentages(data, ['reservation_status'])
display(summary)

,Count,%
reservation_status,,
Canceled,42729,97.3
No-Show,1205,2.7
Check-Out,0,0.0


This is also the case so no cleaning required here.

## Check `is_repeated_guest` matches Previous bookings

### Investigate `previous_bookings_not_canceled`

According to the [original data source](https://www.sciencedirect.com/science/article/pii/S2352340918315191), this variable was assigned as follows:
> In case there was no customer profile associated with the booking, the value is set to 0. Otherwise, the value is the number of bookings with the same customer profile created before the current booking and not canceled.

Presumably this also applies to bookings made before the date range of the current dataset.

View counts

In [ ]:
data = df.drop_duplicates(keep='first')  # Drop duplicates to prevent skewing the data
summary = value_counts_and_percentages(data, ['previous_bookings_not_canceled'])
display(summary)

### Investigate `previous_cancellations`

According to the [original data source](https://www.sciencedirect.com/science/article/pii/S2352340918315191), this variable was assigned as follows:
> In case there was no customer profile associated with the booking, the value is set to 0. Otherwise, the value is the number of bookings with the same customer profile created before the current booking and canceled.

Presumably this also applies to bookings made before the date range of the current dataset.

In [ ]:
data = df.drop_duplicates(keep='first')  # Drop duplicates to prevent skewing the data
summary = value_counts_and_percentages(data, ['previous_cancellations'])
display(summary)

### Investigate `is_repeated_guest`

According to the [original data source](https://www.sciencedirect.com/science/article/pii/S2352340918315191), this variable was

> *"created by verifying if a profile was associated with the booking customer. If so, and if the customer profile creation date was prior to the creation date for the booking on the PMS database it was assumed the booking was from a repeated guest."*

Create a `total_previous_bookings` column

In [ ]:
df_prev_bookings = df.drop_duplicates(keep='first')
df_prev_bookings['total_previous_bookings'] = df_prev_bookings['previous_bookings_not_canceled'] + df_prev_bookings['previous_cancellations']
df_prev_bookings.head(3)


We expect that all records with **is_repeated_guest = 0** will have no previous non-cancelled bookings but this is not found to be the case.

In [ ]:
data = df_prev_bookings[df_prev_bookings['is_repeated_guest']==0]
summary = value_counts_and_percentages(data, ['total_previous_bookings', 'previous_bookings_not_canceled', 'previous_cancellations', 'is_repeated_guest'])
display(summary)

We expect that all records with **is_repeated_guest = 1** will have at least one previous non-cancelled booking but this is not found to be the case either. 

In [ ]:
data = df_prev_bookings[df_prev_bookings['is_repeated_guest']==1]
summary = value_counts_and_percentages(data, ['total_previous_bookings', 'previous_bookings_not_canceled', 'previous_cancellations', 'is_repeated_guest'])
display(summary)

Since the is_repeated_guest feature is derived from whether the PMS had a guest profile created before the booking (rather than using the previous booking data), the discrepancy may have predictive power when training the model.
- ACTION: add an additional feature (an inconsistency flag) during feature engineering and assess feature importance after training the model to see if the signal has any significance.